In [1]:
# Imports
import pandas as pd
from pathlib import Path
from datetime import datetime
import xlsxwriter


In [2]:
df_file_import = pd.read_csv(
    Path("./Resources/Task_SE03.csv")
)

In [3]:
display(df_file_import.head(10))

,Link,Task Name,Pipeline Step,Status,Assigned To
0,PIC_0303_303_005,fx,FX,apr,Vlad Tushevskiy
1,PIC_0303_303_005,comp,Comp,fin,Kimberly Martinez
2,PIC_0303_303_005,anim,Animation,apr,Jason Shulman
3,PIC_0303_303_005,light,Lighting/Rendering,apr,"Forrest Arnold, Patricio Harris"
4,PIC_0303_303_005,ltcompFx,Light Comp,apr,"Bruce Coy, David Silva, Rick Ramirez, Vlad Tus..."
5,PIC_0303_303_005,ltcomp,Light Comp,apr,"Forrest Arnold, Patricio Harris"
6,PIC_0303_303_005,ref,NaN,apr,NaN
7,PIC_0303_303_005,camB,NaN,apr,NaN
8,PIC_0303_303_005,exp,Exports,apr,NaN
9,PIC_0303_303_005,fxNebula,FX,apr,Rick Ramirez


In [4]:
df_file_import.rename({"Pipeline Step" : "category", "Assigned To" : "names"}, axis = 1, inplace = True)
df_file_import_clean = df_file_import.drop(columns = ["Link", "Task Name", "Status"])
df_file_import_clean

,category,names
0,FX,Vlad Tushevskiy
1,Comp,Kimberly Martinez
2,Animation,Jason Shulman
3,Lighting/Rendering,"Forrest Arnold, Patricio Harris"
4,Light Comp,"Bruce Coy, David Silva, Rick Ramirez, Vlad Tus..."
...,...,...
3786,Rigging,Andrei Orehov
3787,Texture,Yinglei Yang
3788,Modeling,"Andrei Orehov, Yinglei Yang"
3789,NaN,NaN


In [5]:
unique_names = [val.strip() for sublist in df_file_import_clean.names.dropna().str.split(",").tolist() for val in sublist]
unique_names = [x.upper() for x in unique_names]
unique_names

['VLAD TUSHEVSKIY',
 'KIMBERLY MARTINEZ',
 'JASON SHULMAN',
 'FORREST ARNOLD',
 'PATRICIO HARRIS',
 'BRUCE COY',
 'DAVID SILVA',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'FORREST ARNOLD',
 'PATRICIO HARRIS',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'STEVE GRAVES',
 'BRUCE COY',
 'VLAD TUSHEVSKIY',
 'VLAD TUSHEVSKIY',
 'DAVID GUTMAN',
 'YINGLEI YANG',
 'DAVID SILVA',
 'VLAD TUSHEVSKIY',
 'DAVID SILVA',
 'VLAD TUSHEVSKIY',
 'YINGLEI YANG',
 'YINGLEI YANG',
 'PETER HERLEIN',
 'JASON SHULMAN',
 'CHRIS PIERZ',
 'DAVID GUTMAN',
 'STEVE GRAVES',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'CHRIS PIERZ',
 'DAVID GUTMAN',
 'RICK RAMIREZ',
 'STEVE GRAVES',
 'YINGLEI YANG',
 'STEVE GRAVES',
 'BRUCE COY',
 'DAVID SILVA',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'STEVE GRAVES',
 'RICK RAMIREZ',
 'VAL KHARITONASHVILI',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'DAVID SILVA',
 'VLAD TUSHEVSKIY',
 'VLAD TUSHEVSKIY',
 'BRUCE COY',
 'VLAD TUSHEVSKIY',
 'BRUC

In [6]:
names_summary = pd.DataFrame(unique_names, columns = ["names"]).value_counts().reset_index().rename(columns = {0:"count"})
names_summary

,names,count
0,RICK RAMIREZ,269
1,YINGLEI YANG,238
2,CHRIS PIERZ,220
3,VLAD TUSHEVSKIY,215
4,WILLIAM MAUER,197
...,...,...
62,KRIS KELLY,1
63,BADNAMED-KIM SYBERG,1
64,KENNEY KIMBLE,1
65,THOMAS MAINE,1


### Excel Formating & Export

Add a table to the excel 

In [7]:
file_name = "./Resources/end_credit.xlsx"
sheet_name = "Summary"

writer = pd.ExcelWriter(file_name, engine = "xlsxwriter")

# writer = pd.ExcelWriter(file_name, engine = "xlsxwriter")

# names_summary.to_excel(writer, sheet_name = sheet_name, startrow = 2, index = False)

Add a title to the excel

In [8]:
workbook = xlsxwriter.Workbook(file_name)
worksheet = workbook.add_worksheet("Summary")

worksheet.write(
    0, 
    0, 
    'Cryptocurrency Pricing Summary on '+datetime.now().strftime('%d %b %Y'), 
    workbook.add_format(
        {'bold': True, 
         'color': '#E26B0A', 
         'size': 14
        }
    )
)


0

Add a remark to the excel

In [9]:
worksheet.write(
    len(names_summary)+4, 
    0, 
    'Remark:', 
    workbook.add_format(
        {'bold': True}
    )
)

worksheet.write(
    len(names_summary)+5, 
    0, 
    'The last update time is ' + datetime.now().strftime('%H:%M') + '.')


0

Add color to the table header

In [10]:
header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'fg_color': '#FDE9D9', 'border': 1})

for col_num, value in enumerate(names_summary.columns.values):
    worksheet.write(2, col_num, value, header_format)


Add a border to the table

In [11]:
row_idx, col_idx = names_summary.shape
for r in range(row_idx):
    for c in range(col_idx):
        if c == 6:
            worksheet.write(r + 3, c, names_summary.values[r, c], workbook.add_format({'border': 1, 'num_format': '0.00%'}))
        else:
            worksheet.write(r + 3, c, names_summary.values[r, c], workbook.add_format({'border': 1, 'num_format': '0.00'}))


Set a column width

In [12]:
worksheet.set_column(0, 6, 12)
worksheet.set_column(1, 1, 20)

0

In [13]:
workbook.close()